In [25]:
from psycopg2.extras import RealDictCursor
import psycopg2
import nltk
import pandas as pd
import re
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Marek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
def create_connection():
    return psycopg2.connect("dbname=twitter user=postgres password=postgres host=localhost port=5433")

In [18]:
add_columns_query = """
ALTER TABLE tweets
ADD COLUMN neg DECIMAL DEFAULT 0,
ADD COLUMN neu DECIMAL DEFAULT 0,
ADD COLUMN pos DECIMAL DEFAULT 0,
ADD COLUMN compound DECIMAL DEFAULT 0
"""

try:
    conn = create_connection();
    cur = conn.cursor()
    cur.execute(add_columns_query)
    conn.commit()
    conn.close()
except:
    conn.rollback()

In [19]:
select_query = """
SELECT DISTINCT t.id, t.content, t.neg, t.neu, t.pos, t.compound
FROM tweets t
JOIN tweet_hashtags th ON th.tweet_id = t.id
JOIN hashtags h ON h.id = th.hashtag_id
WHERE h.value ILIKE any (array[
    '%DeepstateVirus%',
    '%DeepStateFauci%',
    '%DeepStateVaccine%',
    '%QAnon%',
    '%Agenda21%',
    '%CCPVirus%',
    '%ClimateChangeHoax%',
    '%GlobalWarmingHoax%',
    '%ChinaLiedPeopleDied%',
    '%SorosVirus%',
    '%5GCoronavirus%',
    '%MAGA%',
    '%WWG1WGA%',
    '%Chemtrails%',
    '%flatEarth%',
    '%MoonLandingHoax%',
    '%moonhoax%',
    '%illuminati%',
    '%pizzaGateIsReal%',
    '%PedoGateIsReal%',
    '%911truth%',
    '%911insidejob%',
    '%reptilians%'
])
"""

conn = create_connection()
df = pd.read_sql_query(select_query, con=conn)
conn.close()

In [20]:
df.head()

,id,content,neg,neu,pos,compound
0,1034289685227614208,#TrueMAGA〽️🦅\nPEDOPHILIA IS AN EPIDEMIC IN USA...,0.0,0.0,0.0,0.0
1,1037494012033343488,#TrueMAGA 〽️🦅\nPEDOPHILIA ‘EPIDEMIC’ WORLD-UNT...,0.0,0.0,0.0,0.0
2,1038674252940148736,"#USA and #India, Wary of @china, Agree to Stre...",0.0,0.0,0.0,0.0
3,1039279051456831489,#TrueMAGA〽️🦅\nCOMMUNlST CHlNA CENSORSHIP IN AM...,0.0,0.0,0.0,0.0
4,1039284206856286208,#TrueMAGA〽️🦅\nChina owns our dept. The Chinese...,0.0,0.0,0.0,0.0


In [23]:
def remove_emoji(tweet):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
        "]+", re.UNICODE)
    return re.sub(emoji, '', tweet)
    
def remove_mentions_and_hashtags(tweet):
    clarified_tweet = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)", " ", tweet)
    return ' '.join(clarified_tweet.split())

def clarify_tweet(tweet):
    no_emoji = remove_emoji(tweet)
    text = remove_mentions_and_hashtags(no_emoji)
    return sid.polarity_scores(text)

In [26]:
try:
    sid = SentimentIntensityAnalyzer()
    df['vader'] = df['content'].apply(lambda x : clarify_tweet(x))
    df['neg'] = df['vader'].apply(lambda x : x['neg'])
    df['neu'] = df['vader'].apply(lambda x : x['neu'])
    df['pos'] = df['vader'].apply(lambda x : x['pos'])
    df['compound'] = df['vader'].apply(lambda x : x['compound'])
    
except e:
    print(e)

In [27]:
df.head()

,id,content,neg,neu,pos,compound,vader
0,1034289685227614208,#TrueMAGA〽️🦅\nPEDOPHILIA IS AN EPIDEMIC IN USA...,0.000,0.880,0.120,0.5707,"{'neg': 0.0, 'neu': 0.88, 'pos': 0.12, 'compou..."
1,1037494012033343488,#TrueMAGA 〽️🦅\nPEDOPHILIA ‘EPIDEMIC’ WORLD-UNT...,0.097,0.903,0.000,-0.3578,"{'neg': 0.097, 'neu': 0.903, 'pos': 0.0, 'comp..."
2,1038674252940148736,"#USA and #India, Wary of @china, Agree to Stre...",0.000,0.626,0.374,0.6580,"{'neg': 0.0, 'neu': 0.626, 'pos': 0.374, 'comp..."
3,1039279051456831489,#TrueMAGA〽️🦅\nCOMMUNlST CHlNA CENSORSHIP IN AM...,0.195,0.805,0.000,-0.7603,"{'neg': 0.195, 'neu': 0.805, 'pos': 0.0, 'comp..."
4,1039284206856286208,#TrueMAGA〽️🦅\nChina owns our dept. The Chinese...,0.145,0.750,0.105,-0.3802,"{'neg': 0.145, 'neu': 0.75, 'pos': 0.105, 'com..."


In [46]:
df[(df['neg'] == 0) & (df['neu'] == 0) & (df['pos'] == 0)]

,id,content,neg,neu,pos,compound,vader
4875,1223282357576269824,#coronavirus #PrayForChina #EconomicSurvey #MA...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
5853,1225455225307156481,@KashJackson2018 #FakeNews #coronavirus #MAGAts,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
8289,1229813655119765504,@VincentCrypt46 #CoronaVirus \n#WWG1WGAworldwide,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
8519,1230547754847526918,@QNNTexas #Coronavirus\n\n#MAGA #KAG #Trump202...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
8988,1232263718194601985,@Qanon76 @realDonaldTrump @TheCollectiveQ #WW...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
12656,1233812604331520003,@Pontifex #RareDiseaseDay \n#coronavirus \n\n#...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
13014,1245223309651767296,@DLoesch @heathrodgirs #ChineseVirus19\n#Chine...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
13105,1245294767904350210,@AskAnshul #chinesevirus #ChinaLiedPeopleDied ...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
13106,1245295036289503232,@amitmalviya #ChinaLiedPeopleDied #ChineseViru...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
13131,1245323194589995009,@WHO @WHOThailand @WHOSEARO @WHOWPRO #ChinaLie...,0.0,0.0,0.0,0.0,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."


In [34]:
try:
    conn = create_connection()
    cur = conn.cursor()

    for index, row in df.iterrows():
        update_query = """
        UPDATE tweets
        SET neg = {},
            neu = {},
            pos = {},
            compound = {}
        WHERE '{}' = tweets.id;
        """.format(row['neg'], row['neu'], row['pos'], row['compound'], row['id'])
        cur.execute(update_query)

    conn.commit()
    conn.close()
    
except:
    pass